# Core Plugins

![](../images/plugins.png)

- We want to have a vat of plugins ... and then find the right plugin to fit the goal ...

- You can find more about the predefined plugins used below [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/out-of-the-box-plugins?tabs=Csharp).


# Setup

## Python

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.planning import ActionPlanner
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill

from IPython.display import display, Markdown

## Kernel

In [ ]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()

kernel.add_text_completion_service(
    "openaicompletion", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

kernel.add_text_embedding_generation_service(
    "openaiembedding", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

# Add Core Plugins


![](/images/core_plugins.png)

## Import plugins


- Action Planner is a simple tool which can find the right tool to get things done

In [ ]:
planner = ActionPlanner(kernel)

- Adding the tools for the kernel to do math, to read/write files, to tell the time, and to play with text:



In [ ]:
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

# Example Plugins

## Math

In [ ]:
ask = "What is the sum of 110 and 990?"



- Finding the most similar function available to get that done...

In [ ]:
plan = await planner.create_plan_async(goal=ask)

In [ ]:
print(
    f" The best single function to use is `{plan._skill_name}.{plan._function.name}`")

## Date {.smaller}

In [ ]:
ask = "What is today?"

In [ ]:
print(f" Finding the most similar function available to get that done...")

In [ ]:
plan = await planner.create_plan_async(goal=ask)

In [ ]:
print(
    f" The best single function to use is `{plan._skill_name}.{plan._function.name}`")

## Text

In [ ]:
ask = "How do I write the word 'text' to a file?"

In [ ]:
print(f" Finding the most similar function available to get that done...")
plan = await planner.create_plan_async(goal=ask)
print(
    f" The best single function to use is `{plan._skill_name}.{plan._function.name}`")

# Text example Poem

## Custom Plugin LiterateFriend

In [ ]:
plugins_directory = "../plugins-sk"

In [ ]:
writer_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "LiterateFriend")

## Use Sequential Planner

In [ ]:
planner = SequentialPlanner(kernel)

## Task

In [ ]:
ask = """
Tomorrow is Valentine's day. I need to come up with a poem. Translate the poem to German.
"""

- This will require two plugins (write poem and translate)

## Call planner

In [ ]:
plan = await planner.create_plan_async(goal=ask)

## Obtain result

In [ ]:
result = await plan.invoke_async()

## Output

In [ ]:
for index, step in enumerate(plan._steps):
    print(f"✅ Step {index+1} used function `{step._function.name}`")

trace_resultp = True

display(
    Markdown(f"## ✨ Generated result from the ask: {ask}\n\n---\n" + str(result)))

## Display output

```markdown
✅ Step 1 used function `ShortPoem`
✅ Step 2 used function `Translate`
✨ Generated result from the ask:

Tomorrow is Valentine's day. I need to come up with a poem. Translate the poem to German.

Rosen sind rot, Veilchen sind blau, Valentinstag ist da, was wirst du tun? Kaufe Schokolade und Blumen oder schreibe einen Liebesbrief, Vergiss nur nicht, es alles besser zu machen.

Wenn du Single bist, keine Sorge, Gönn dir selbst, zu...
```

- mmmhhh, maybe lost in translation 🙂


## Take a closer look at the output

In [ ]:
for index, step in enumerate(plan._steps):
    print(f"✅ Step {index+1} used function `{step._function.name}`")

trace_resultp = True

if trace_resultp:
    print("Longform trace:\n")
    for index, step in enumerate(plan._steps):
        print("Step:", index)
        print("Description:", step.description)
        print("Function:", step.skill_name + "." + step._function.name)
        print("Output vars:", step._outputs)
        if len(step._outputs) > 0:
            print("  Output:\n", str.replace(
                result[step._outputs[0]], "\n", "\n  "))

display(
    Markdown(f"## ✨ Generated result from the ask: {ask}\n\n---\n" + str(result)))